# Описание проекта

В качестве экзаменационного задания предложен аналитический кейс, в котором используются данные о посещаемости сайта datalytics.ru, а также искусственно сгенерированные данные о доходах, полученных от рекламы на сайте.
Необходимо выяснить, пользователи каких устройств приносят владельцу сайта наибольший  доход.
Для этого соберем данные о доходах и данные об аудитории сайта за май 2019 года в единый отчёт, посчитаем метрику ARPU (average revenue per user) по каждому устройству и найдем модель устройства с наибольшим ARPU.
NB При расчёте суточной аудитории учитываем только те устройства, которыми в течение месяца воспользовались не менее трёх посетителей сайта.

### Описание данных

- revenue_data.csv содержит сведения о доходах от рекламы в разбивке по моделям устройств. В файле просуммирован общий доход (revenue) по каждому устройству за май 2019 года.
- Данные Яндекс.Метрики о суточной аудитории сайта datalytics.ru в разбивке по устройствам

### Результат решения кейса:

Понять конкретную модель устройства, пользователи которой принесли наибольший доход, переходя по рекламным ссылкам на сайте.

In [2]:
import requests  
import json  
import datetime  
import time  
import schedule  
import pandas as pd

# Извлечем данные Яндекс.Метрики о суточной аудитории сайта datalytics.ru в разбивке по устройствам
# по API с помощью токена.

token = 'AgAAAAAaLVoAAAbDZUcioA1lNU51gucoxFkaD5w'
headers = {'Authorization': 'OAuth ' + token}
params = {'metrics': 'ym:s:users', 
          'dimensions': 'ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel,ym:s:date',  
          'date1': '2019-05-01',  
          'date2': '2019-05-31',
          'limit': 1000,
          'ids': 30177909}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code) # Проверяем отклик

200


In [3]:
metrika_data = response.json()
print (metrika_data.keys())

dict_keys(['query', 'data', 'total_rows', 'total_rows_rounded', 'sampled', 'contains_sensitive_data', 'sample_share', 'sample_size', 'sample_space', 'data_lag', 'totals', 'min', 'max'])


In [4]:
print (metrika_data['total_rows'])

320


In [5]:
metrika_data['data']

[{'dimensions': [{'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'PC',
    'id': 'desktop'},
   {'icon_id': None, 'icon_type': None, 'name': None, 'id': None},
   {'icon_type': None, 'name': None, 'icon_id': None},
   {'name': '2019-05-20'}],
  'metrics': [105.0]},
 {'dimensions': [{'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'PC',
    'id': 'desktop'},
   {'icon_id': None, 'icon_type': None, 'name': None, 'id': None},
   {'icon_type': None, 'name': None, 'icon_id': None},
   {'name': '2019-05-16'}],
  'metrics': [94.0]},
 {'dimensions': [{'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'PC',
    'id': 'desktop'},
   {'icon_id': None, 'icon_type': None, 'name': None, 'id': None},
   {'icon_type': None, 'name': None, 'icon_id': None},
   {'name': '2019-05-22'}],
  'metrics': [94.0]},
 {'dimensions': [{'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'PC',
    'id': 'desktop'},
   {'icon_id': None, 'icon_type': None, 'name': None, '

In [6]:
result = [] #Создаём  список куда вносим словари в новом формате

for data_item in metrika_data['data']: #Перебираем элементы старого отчёта 
    new_dict = {}  #Создаём словарь для хранения текущих данных в новом формате
    for i,dimension in enumerate(data_item['dimensions']):  #Обращаемся к разделу отчёта, содержащего метаданные и перебираем названия группировок
        new_dict[metrika_data['query']['dimensions'][i]] = dimension['name'] # Создаём в новом словаре ключ для каждой группировки, устанавливаем значение для каждого ключа
        
    for i,metric in enumerate(data_item['metrics']): # Те же действия выполняем для метрик
        new_dict[metrika_data['query']['metrics'][i]] = metric
    result.append(new_dict) # Добавляем созданный словарь в итоговый список

In [7]:
# Преобразуем в df
res = pd.DataFrame(result)

In [8]:
# Переименовываем колонки
res.columns=['device_category', 'mobile_phone', 'mobile_phone_model', 'date', 'qty_user']

In [9]:
# Избавимся от пропусков
res = res.dropna() 
res = res.reset_index(drop=True)
res

,device_category,mobile_phone,mobile_phone_model,date,qty_user
0,Smartphones,Apple,iPhone,2019-05-16,10.0
1,Smartphones,Apple,iPhone,2019-05-07,7.0
2,Smartphones,Apple,iPhone,2019-05-15,6.0
3,Smartphones,Apple,iPhone,2019-05-18,6.0
4,Smartphones,Apple,iPhone,2019-05-06,5.0
...,...,...,...,...,...
275,Tablets,Samsung,Galaxy Tab A 9.7,2019-05-10,1.0
276,Tablets,Samsung,Galaxy Tab E 9.6,2019-05-15,1.0
277,Tablets,Huawei,T1-701u,2019-05-24,1.0
278,Tablets,Digma,CITI 1903 4G CS1062ML,2019-05-24,1.0


In [10]:
# группируем данные и суммируем кол-во пользователей и 
# фильтруем по кол-ву пользователей больше 3
res = res.groupby(['device_category','mobile_phone','mobile_phone_model']).sum()  
res = res[res['qty_user']>=3]
res

qty_user
device_category mobile_phone mobile_phone_model          
Smartphones     ASUS         ZC554KL                  3.0
                Apple        iPhone                 118.0
                Huawei       AUM-L29                  3.0
                             CLT-L29                  3.0
                             COL-L29                  3.0
                             LLD-L31                  4.0
                             STF-L09                  5.0
                Meizu        16th                     3.0
                Samsung      Galaxy A5                7.0
                             Galaxy J7                3.0
                             Galaxy S7                5.0
                             Galaxy S8                6.0
                             SAMSUNG SM-G935W8        3.0
                             SM-G965F                 4.0
                             SM-N960F                 4.0
                Sony         G3112                    3.0
                Xiaomi       MI 8                     3.0
                             MI 8 SE                  3.0
                             Mi A1                    7.0
                             Mi A2 Lite               3.0
                             Redmi 4X                 5.0
                             Redmi 5 Plus             3.0
                             Redmi 5A                 3.0
                             Redmi Note 3             3.0
                             Redmi Note 4             7.0
                             Redmi Note 4X            4.0
                             Redmi Note 5             7.0
                             Redmi Note 5A            3.0
Tablets         Apple        iPad                    13.0

In [11]:
# проводим проверку и предобработку данных
df = pd.read_csv('revenue_data.csv', sep ='\t', decimal = ',') 
df['revenue'] = df['revenue'].astype(float)
df.columns=['device_category', 'mobile_phone', 'mobile_phone_model', 'revenue']
df

,device_category,mobile_phone,mobile_phone_model,revenue
0,PC,None,None,1883.870896
1,Smartphones,ASUS,ZB500KL,0.520907
2,Smartphones,ASUS,ZB555KL,0.434003
3,Smartphones,ASUS,ZB602KL,0.317529
4,Smartphones,ASUS,ZC554KL,1.130402
...,...,...,...,...
141,Tablets,None,800P11B,0.935338
142,Tablets,Samsung,Galaxy Tab 3 7.0 Lite,1.013974
143,Tablets,Samsung,Galaxy Tab A 9.7,1.034228
144,Tablets,Samsung,Galaxy Tab E 9.6,0.687359


In [12]:
#объединяем df по идентичным колонкам  
# ищем ARPU для всех девайсов
joined = df.merge(res, on=['device_category','mobile_phone','mobile_phone_model'])
joined['ARPU'] = joined['revenue']/joined['qty_user']
joined_df = joined.groupby(['mobile_phone_model'])['ARPU'].max().sort_values(ascending=False)


In [13]:
joined_df.head(1) 

mobile_phone_model
Redmi Note 5A    1.413608
Name: ARPU, dtype: float64

### Вывод:

Пользовальзователи Redmi Note 5A принесли найбольшиий доход за рекламу.